# GRPO Reranking Kiértékelése - CourtRankRL Projekt

Ez a notebook a CourtRankRL GRPO alapú reranking komponensét értékeli ki. Az agents.md specifikáció alapján a reinforcement learning reranking teljesítményét és hatékonyságát teszteli.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import torch
from pathlib import Path
from typing import Dict, Any, List
import time

# Plot stílus beállítása
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Projekt konfiguráció betöltése
import sys
project_root = Path(__file__).parent.parent
sys.path.insert(0, str(project_root))
from configs import config
from src.search.hybrid_search import HybridRetriever
from src.search.grpo_reranker import GRPOReranker

print("CourtRankRL - GRPO Reranking Evaluation")
print(f"RL Policy: {config.RL_POLICY_PATH}")
print(f"BM25 index: {config.BM25_INDEX_PATH}")
print(f"FAISS index: {config.FAISS_INDEX_PATH}")

## 1. Komponensek Betöltése

A GRPO reranker és retrieval komponensek betöltése.

In [ ]:
# Komponensek inicializálása
retriever = None
reranker = None
policy_loaded = False

print("Komponensek betöltése...")

# Hybrid retriever
try:
    retriever = HybridRetriever()
    print("✅ Hybrid retriever betöltve")
except Exception as e:
    print(f"❌ Hybrid retriever hiba: {e}")
    retriever = None

# GRPO reranker
try:
    reranker = GRPOReranker()
    print("✅ GRPO reranker inicializálva")
    
    # Policy betöltése
    if config.RL_POLICY_PATH.exists():
        reranker.load_policy(config.RL_POLICY_PATH)
        policy_loaded = True
        print("✅ RL policy betöltve")
    else:
        print(f"⚠️ RL policy nem található: {config.RL_POLICY_PATH}")
        print("Policy nélküli reranker működés (baseline)")
        
except Exception as e:
    print(f"❌ GRPO reranker hiba: {e}")
    reranker = None

if reranker is not None:
    print(f"\nReranker konfiguráció:")
    print(f"  Learning rate: {config.RL_LEARNING_RATE}")
    print(f"  Batch size: {config.RL_BATCH_SIZE}")
    print(f"  Hidden dim: {config.RL_HIDDEN_DIM}")
    print(f"  Policy loaded: {policy_loaded}")

## 2. Reranking Teljesítmény Tesztelése

A GRPO reranking teljesítményének és hatékonyságának tesztelése.

In [ ]:
if retriever is not None and reranker is not None:
    print("🎯 Reranking teljesítmény tesztelése:")
    
    # Teszt lekérdezések
    test_queries = [
        "szerződés felmondása",
        "kártérítés",
        "családi jog",
        "munkajog",
        "ingatlan tulajdonjog"
    ]
    
    results_summary = []
    
    for query in test_queries:
        print(f"\n🔍 Teszt lekérdezés: '{query}'")
        
        try:
            # Baseline retrieval
            start_time = time.time()
            bm25_results, dense_results = retriever.retrieve_candidates(query, top_k=config.TOP_K_BASELINE)
            baseline_time = time.time() - start_time
            
            print(f"  Baseline retrieval: {len(bm25_results)} BM25 + {len(dense_results)} dense = {len(bm25_results) + len(dense_results)} candidate")
            print(f"  Baseline idő: {baseline_time*1000:.1f}ms")
            
            # Baseline fusion (RRF)
            start_time = time.time()
            baseline_fusion = retriever.retrieve(query, top_k=config.TOP_K_RERANKED, fusion_method="rrf")
            fusion_time = time.time() - start_time
            
            print(f"  Baseline fusion (RRF): {len(baseline_fusion)} eredmény, {fusion_time*1000:.1f}ms")
            
            # GRPO reranking
            start_time = time.time()
            try:
                reranked_results = reranker.rerank(bm25_results, dense_results)
                reranking_time = time.time() - start_time
                
                print(f"  GRPO reranking: {len(reranked_results)} eredmény, {reranking_time*1000:.1f}ms")
                
                # Eredmények összehasonlítása
                if reranked_results and baseline_fusion:
                    # Átfedés számítása
                    reranked_set = set([doc_id for doc_id, _ in reranked_results[:5]])
                    baseline_set = set(baseline_fusion[:5])
                    
                    overlap = len(reranked_set & baseline_set) / len(baseline_set) if baseline_set else 0
                    print(f"  Átfedés baseline vs reranked (top 5): {overlap:.2f}")
                    
                    # Eltérések
                    only_reranked = reranked_set - baseline_set
                    only_baseline = baseline_set - reranked_set
                    
                    if only_reranked:
                        print(f"  Csak reranked-ben: {list(only_reranked)}")
                    if only_baseline:
                        print(f"  Csak baseline-ben: {list(only_baseline)}")
                    
                    # Reranked eredmények megjelenítése
                    print(f"\nTop 5 reranked eredmény:")
                    for i, (doc_id, score) in enumerate(reranked_results[:5], 1):
                        print(f"  {i}. {doc_id} (score: {score:.4f})")
                
                # Összefoglaló adatok
                results_summary.append({
                    'query': query,
                    'baseline_results': len(baseline_fusion),
                    'baseline_time': baseline_time * 1000,
                    'fusion_time': fusion_time * 1000,
                    'reranking_time': reranking_time * 1000,
                    'total_reranking_time': (baseline_time + reranking_time) * 1000,
                    'overlap_ratio': overlap if 'overlap' in locals() else 0
                })
                
            except Exception as rerank_e:
                print(f"❌ Reranking hiba: {rerank_e}")
                results_summary.append({
                    'query': query,
                    'baseline_results': len(baseline_fusion),
                    'baseline_time': baseline_time * 1000,
                    'fusion_time': fusion_time * 1000,
                    'reranking_time': 0,
                    'total_reranking_time': baseline_time * 1000,
                    'overlap_ratio': 0
                })
            
        except Exception as e:
            print(f"❌ Teszt hiba: {e}")
    
    # Összefoglaló táblázat
    if results_summary:
        results_df = pd.DataFrame(results_summary)
        print("\n📊 Reranking teljesítmény összefoglaló:")
        display(results_df.round(2))
        
        # Átlagos teljesítmény
        print("\n📈 Átlagos teljesítmény:")
        print(f"  Baseline retrieval: {results_df['baseline_time'].mean():.1f}ms átlag")
        print(f"  Fusion: {results_df['fusion_time'].mean():.1f}ms átlag")
        print(f"  Reranking: {results_df['reranking_time'].mean():.1f}ms átlag")
        print(f"  Összes reranking: {results_df['total_reranking_time'].mean():.1f}ms átlag")
        print(f"  Átfedés arány: {results_df['overlap_ratio'].mean():.2f} átlag")
        
        # Teljesítmény vizualizáció
        plt.figure(figsize=(14, 6))
        
        x = np.arange(len(test_queries))
        width = 0.2
        
        plt.bar(x - width, results_df['baseline_time'], width, label='Baseline', alpha=0.7)
        plt.bar(x, results_df['fusion_time'], width, label='Fusion', alpha=0.7)
        plt.bar(x + width, results_df['reranking_time'], width, label='Reranking', alpha=0.7)
        
        plt.xlabel('Teszt lekérdezés')
        plt.ylabel('Idő (ms)')
        plt.title('Reranking teljesítmény összehasonlítása')
        plt.xticks(x, [f'Q{i+1}' for i in range(len(test_queries))], rotation=45)
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
else:
    print("❌ Reranking tesztelés nem elérhető - hiányzó komponensek")
    if retriever is None:
        print("💡 Szükséges: Hybrid retriever - futtassa: uv run courtrankrl build")
    if reranker is None:
        print("💡 Szükséges: GRPO reranker - ellenőrizze a GRPOReranker implementációt")

## 3. Policy Elemzése

A betanított RL policy tulajdonságainak és teljesítményének elemzése.

In [ ]:
if reranker is not None and policy_loaded:
    print("🧠 Policy elemzése:")
    
    try:
        # Policy tulajdonságok
        policy_info = reranker.get_policy_info()
        
        print(f"Policy információk:")
        print(f"  Policy típusa: {policy_info.get('type', 'N/A')}")
        print(f"  Input dimenzió: {policy_info.get('input_dim', 'N/A')}")
        print(f"  Hidden dimenzió: {policy_info.get('hidden_dim', config.RL_HIDDEN_DIM)}")
        print(f"  Paraméterek száma: {policy_info.get('param_count', 'N/A'):,}")
        print(f"  Tanítási idő: {policy_info.get('training_time', 'N/A')}")
        
        # Policy teljesítmény metrikák
        if 'metrics' in policy_info:
            print(f"\nTanítási metrikák:")
            metrics = policy_info['metrics']
            for epoch, epoch_metrics in metrics.items():
                print(f"  Epoch {epoch}:")
                for metric, value in epoch_metrics.items():
                    print(f"    {metric}: {value:.4f}")
        
        # Feature súlyok elemzése (ha lineáris policy)
        if 'feature_weights' in policy_info:
            print(f"\nFeature súlyok:")
            weights = policy_info['feature_weights']
            
            # Súlyok vizualizációja
            plt.figure(figsize=(10, 6))
            features = list(weights.keys())
            weight_values = list(weights.values())
            
            plt.barh(features, weight_values)
            plt.title('Feature súlyok a policy-ben')
            plt.xlabel('Súly')
            plt.ylabel('Feature')
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
            
            # Legfontosabb feature-ök
            sorted_weights = sorted(weights.items(), key=lambda x: abs(x[1]), reverse=True)
            print(f"\nLegfontosabb feature-ök:")
            for i, (feature, weight) in enumerate(sorted_weights[:5]):
                print(f"  {i+1}. {feature}: {weight:.4f}")
        
        # Policy stabilitás
        if 'stability_metrics' in policy_info:
            print(f"\nPolicy stabilitás:")
            stability = policy_info['stability_metrics']
            for metric, value in stability.items():
                print(f"  {metric}: {value:.4f}")
                
    except Exception as e:
        print(f"❌ Policy elemzés hiba: {e}")
        print("Policy metrikák nem elérhetőek")
else:
    print("❌ Policy elemzés nem elérhető - hiányzó policy")
    if not policy_loaded:
        print("💡 Policy betöltése szükséges: futtassa a train parancsot")

## 4. Feature Elemzése

A GRPO reranking által használt feature-ök elemzése.

In [ ]:
if retriever is not None and reranker is not None:
    print("📊 Feature elemzése:")
    
    # Teszt lekérdezés feature elemzéshez
    test_query = "szerződés felmondása"
    
    try:
        # Kandidátusok lekérése
        bm25_results, dense_results = retriever.retrieve_candidates(test_query, top_k=10)
        
        if bm25_results and dense_results:
            # Features számítása
            features = reranker.extract_features(bm25_results, dense_results, test_query)
            
            print(f"\nTeszt lekérdezés: '{test_query}'")
            print(f"Feature mátrix: {features.shape}")
            
            # Feature statisztikák
            feature_stats = pd.DataFrame({
                'mean': features.mean(axis=0),
                'std': features.std(axis=0),
                'min': features.min(axis=0),
                'max': features.max(axis=0)
            })
            
            print(f"\nFeature statisztikák:")
            display(feature_stats.round(4))
            
            # Feature korreláció
            if features.shape[1] > 1:
                correlation_matrix = np.corrcoef(features.T)
                
                plt.figure(figsize=(8, 6))
                sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
                          xticklabels=[f'F{i}' for i in range(features.shape[1])],
                          yticklabels=[f'F{i}' for i in range(features.shape[1])])
                plt.title('Feature korreláció mátrix')
                plt.tight_layout()
                plt.show()
            
            # Feature eloszlások
            plt.figure(figsize=(14, 8))
            for i in range(min(6, features.shape[1])):  # Maximum 6 feature
                plt.subplot(2, 3, i+1)
                plt.hist(features[:, i], bins=20, alpha=0.7)
                plt.title(f'Feature {i} eloszlása')
                plt.xlabel(f'Feature {i} érték')
                plt.ylabel('Kandidátusok száma')
                plt.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
            
            # Policy score-ok
            if policy_loaded:
                try:
                    scores = reranker.policy.predict(features)
                    
                    print(f"\nPolicy score statisztikák:")
                    print(f"  Átlag score: {scores.mean():.4f}")
                    print(f"  Score tartomány: [{scores.min():.4f}, {scores.max():.4f}]")
                    
                    # Score és feature kapcsolat
                    print(f"\nTop 5 scoring kandidátus:")
                    top_indices = np.argsort(scores)[::-1][:5]
                    for i, idx in enumerate(top_indices):
                        doc_id = bm25_results[idx] if idx < len(bm25_results) else dense_results[idx - len(bm25_results)]
                        print(f"  {i+1}. {doc_id}: {scores[idx]:.4f}")
                        print(f"      Features: {features[idx]}")
                        
                except Exception as score_e:
                    print(f"❌ Score számítás hiba: {score_e}")
        
    except Exception as e:
        print(f"❌ Feature elemzés hiba: {e}")
else:
    print("❌ Feature elemzés nem elérhető - hiányzó komponensek")

## 5. Ranking Összehasonlítás

A baseline és GRPO reranked eredmények összehasonlítása.

In [ ]:
if retriever is not None and reranker is not None and policy_loaded:
    print("📊 Ranking összehasonlítás:")
    
    # Teszt lekérdezések részletes elemzéshez
    test_queries_detailed = [
        "szerződés felmondása",
        "kártérítés mértéke",
        "családjogi ügyek"
    ]
    
    comparison_results = []
    
    for query in test_queries_detailed:
        try:
            # Baseline retrieval
            baseline_results = retriever.retrieve(query, top_k=config.TOP_K_RERANKED, fusion_method="rrf")
            
            # Reranked retrieval
            bm25_results, dense_results = retriever.retrieve_candidates(query, top_k=config.TOP_K_BASELINE)
            reranked_results = reranker.rerank(bm25_results, dense_results)
            
            if baseline_results and reranked_results:
                # Összehasonlítás
                baseline_set = set(baseline_results[:5])
                reranked_set = set([doc_id for doc_id, _ in reranked_results[:5]])
                
                overlap = len(baseline_set & reranked_set) / len(baseline_set) if baseline_set else 0
                
                print(f"\n🔍 Lekérdezés: '{query}'")
                print(f"\nBaseline ranking:")
                for i, doc_id in enumerate(baseline_results[:5], 1):
                    print(f"  {i}. {doc_id}")
                
                print(f"\nReranked ranking:")
                for i, (doc_id, score) in enumerate(reranked_results[:5], 1):
                    print(f"  {i}. {doc_id} (score: {score:.4f})")
                
                print(f"\nÁtfedés (top 5): {overlap:.2f}")
                
                # Kendall tau korreláció (ha lehetséges)
                try:
                    from scipy.stats import kendalltau
                    
                    # Közös dokumentumok rangsorolása
                    common_docs = list(baseline_set & reranked_set)
                    if len(common_docs) >= 3:  # Minimum 3 dokumentum a korrelációhoz
                        baseline_ranks = [baseline_results.index(doc) for doc in common_docs]
                        reranked_ranks = [reranked_results.index((doc, 0)) for doc in common_docs]
                        
                        tau, p_value = kendalltau(baseline_ranks, reranked_ranks)
                        print(f"  Kendall tau korreláció: {tau:.3f} (p-value: {p_value:.3f})")
                    
                except ImportError:
                    print("  (scipy nem elérhető a Kendall tau korrelációhoz)")
                except Exception as kendall_e:
                    print(f"  Kendall tau számítás hiba: {kendall_e}")
                
                comparison_results.append({
                    'query': query,
                    'overlap': overlap,
                    'baseline_top1': baseline_results[0] if len(baseline_results) > 0 else None,
                    'reranked_top1': reranked_results[0][0] if len(reranked_results) > 0 else None
                })
            
        except Exception as e:
            print(f"❌ Összehasonlítás hiba: {e}")
    
    # Összefoglaló
    if comparison_results:
        comparison_df = pd.DataFrame(comparison_results)
        print(f"\n📋 Összehasonlítás összefoglaló:")
        display(comparison_df)
        
        print(f"\nÁtlagos átfedés: {comparison_df['overlap'].mean():.2f}")
        
        # Baseline vs reranked top-1 változás
        top1_changes = comparison_df['baseline_top1'] != comparison_df['reranked_top1']
        print(f"Top-1 változás: {top1_changes.sum()}/{len(comparison_df)} lekérdezés")
else:
    print("❌ Ranking összehasonlítás nem elérhető - hiányzó komponensek vagy policy")

## 6. Training Data Elemzése

A GRPO training során használt adatok és folyamat elemzése.

In [ ]:
if reranker is not None:
    print("🎓 Training data elemzése:")
    
    try:
        # Training data információk
        training_info = reranker.get_training_info()
        
        if training_info:
            print(f"Training információk:")
            print(f"  Qrels fájl: {training_info.get('qrels_file', 'N/A')}")
            print(f"  Training minták: {training_info.get('num_samples', 'N/A')}")
            print(f"  Epoch-ok: {training_info.get('epochs', 'N/A')}")
            print(f"  Batch size: {training_info.get('batch_size', config.RL_BATCH_SIZE)}")
            
            # Training metrikák
            if 'training_metrics' in training_info:
                print(f"\nTraining metrikák:")
                metrics = training_info['training_metrics']
                
                # Metrikák vizualizációja
                if isinstance(metrics, dict) and 'epochs' in metrics:
                    epochs = metrics['epochs']
                    
                    plt.figure(figsize=(14, 8))
                    
                    # Loss görbe
                    if 'loss' in metrics:
                        plt.subplot(2, 2, 1)
                        plt.plot(epochs, metrics['loss'])
                        plt.title('Training Loss')
                        plt.xlabel('Epoch')
                        plt.ylabel('Loss')
                        plt.grid(True, alpha=0.3)
                    
                    # Reward görbe
                    if 'reward' in metrics:
                        plt.subplot(2, 2, 2)
                        plt.plot(epochs, metrics['reward'])
                        plt.title('Average Reward')
                        plt.xlabel('Epoch')
                        plt.ylabel('Reward')
                        plt.grid(True, alpha=0.3)
                    
                    # nDCG görbe
                    if 'ndcg' in metrics:
                        plt.subplot(2, 2, 3)
                        plt.plot(epochs, metrics['ndcg'])
                        plt.title('nDCG Score')
                        plt.xlabel('Epoch')
                        plt.ylabel('nDCG')
                        plt.grid(True, alpha=0.3)
                    
                    # Policy entropy
                    if 'entropy' in metrics:
                        plt.subplot(2, 2, 4)
                        plt.plot(epochs, metrics['entropy'])
                        plt.title('Policy Entropy')
                        plt.xlabel('Epoch')
                        plt.ylabel('Entropy')
                        plt.grid(True, alpha=0.3)
                    
                    plt.tight_layout()
                    plt.show()
                
                # Legutolsó metrikák
                print(f"\nLegutolsó epoch metrikák:")
                for metric, values in metrics.items():
                    if isinstance(values, list) and values:
                        print(f"  {metric}: {values[-1]:.4f}")
                    elif not isinstance(values, list):
                        print(f"  {metric}: {values:.4f}")
            
            # Qrels statisztikák
            if 'qrels_stats' in training_info:
                qrels_stats = training_info['qrels_stats']
                print(f"\nQrels statisztikák:")
                for stat, value in qrels_stats.items():
                    print(f"  {stat}: {value}")
        else:
            print("⚠️ Training információk nem elérhetőek")
            print("A policy betanítása után lesznek elérhetőek a részletes metrikák")
    
    except Exception as e:
        print(f"❌ Training data elemzés hiba: {e}")
        print("Training metrikák nem elérhetőek")
else:
    print("❌ Training data elemzés nem elérhető - hiányzó reranker")

## 7. Ablation Study

A GRPO komponensek hatásának vizsgálata.

In [ ]:
if retriever is not None and reranker is not None:
    print("🔬 Ablation study:")
    
    # Teszt lekérdezés
    test_query = "szerződés felmondása"
    
    try:
        # Összes komponens bekapcsolva
        bm25_results, dense_results = retriever.retrieve_candidates(test_query, top_k=config.TOP_K_BASELINE)
        
        # Baseline (csak RRF fusion)
        baseline_results = retriever.retrieve(test_query, top_k=config.TOP_K_RERANKED, fusion_method="rrf")
        
        # Reranking (BM25 + dense + policy)
        if policy_loaded:
            reranked_results = reranker.rerank(bm25_results, dense_results)
            
            # Csak BM25 (ablation)
            bm25_only_results = reranker.rerank(bm25_results[:10], [])  # Üres dense lista
            
            # Csak dense (ablation)
            dense_only_results = reranker.rerank([], dense_results[:10])  # Üres BM25 lista
            
            print(f"\nAblation study - '{test_query}':")
            
            # Eredmények összehasonlítása
            ablation_results = {
                'Baseline (RRF)': baseline_results[:5],
                'Reranked (full)': [doc_id for doc_id, _ in reranked_results[:5]],
                'BM25 only': [doc_id for doc_id, _ in bm25_only_results[:5]],
                'Dense only': [doc_id for doc_id, _ in dense_only_results[:5]]
            }
            
            # Átfedések számítása
            baseline_set = set(ablation_results['Baseline (RRF)'])
            reranked_set = set(ablation_results['Reranked (full)'])
            bm25_set = set(ablation_results['BM25 only'])
            dense_set = set(ablation_results['Dense only'])
            
            print(f"\nÁtfedések (top 5):")
            print(f"  Baseline vs Reranked: {len(baseline_set & reranked_set)}/5")
            print(f"  Baseline vs BM25: {len(baseline_set & bm25_set)}/5")
            print(f"  Baseline vs Dense: {len(baseline_set & dense_set)}/5")
            print(f"  BM25 vs Dense: {len(bm25_set & dense_set)}/5")
            
            # Eredmények megjelenítése
            print(f"\nEredmények:")
            for method, results in ablation_results.items():
                print(f"\n{method}:")
                for i, doc_id in enumerate(results, 1):
                    print(f"  {i}. {doc_id}")
            
            # Ablation metrikák
            print(f"\nAblation metrikák:")
            print(f"  Baseline diverzitás: {len(baseline_set)} egyedi dokumentum")
            print(f"  Reranked diverzitás: {len(reranked_set)} egyedi dokumentum")
            print(f"  BM25 diverzitás: {len(bm25_set)} egyedi dokumentum")
            print(f"  Dense diverzitás: {len(dense_set)} egyedi dokumentum")
            
            # Kombináció előnye
            combo_advantage = len(baseline_set & reranked_set) / len(baseline_set) if baseline_set else 0
            print(f"  Kombináció előnye: {combo_advantage:.2f} (baseline vs reranked átfedés)")
        else:
            print("⚠️ Ablation study policy nélkül nem teljes - csak baseline vs fusion összehasonlítás")
            
            print(f"\nBaseline vs Fusion összehasonlítás:")
            baseline_results = retriever.retrieve(test_query, top_k=config.TOP_K_RERANKED, fusion_method="rrf")
            
            print(f"\nBaseline (RRF) eredmények:")
            for i, doc_id in enumerate(baseline_results[:5], 1):
                print(f"  {i}. {doc_id}")
            
            # Egyszerű fusion vs baseline összehasonlítás
            bm25_results, dense_results = retriever.retrieve_candidates(test_query, top_k=20)
            
            bm25_set = set(bm25_results[:5])
            dense_set = set(dense_results[:5])
            baseline_set = set(baseline_results[:5])
            
            print(f"\nBM25 top 5: {list(bm25_set)}")
            print(f"Dense top 5: {list(dense_set)}")
            
            print(f"\nÁtfedések:")
            print(f"  BM25 vs Baseline: {len(bm25_set & baseline_set)}/5")
            print(f"  Dense vs Baseline: {len(dense_set & baseline_set)}/5")
    
    except Exception as e:
        print(f"❌ Ablation study hiba: {e}")
else:
    print("❌ Ablation study nem elérhető - hiányzó komponensek")

## 8. Következtetések

A GRPO reranking kiértékelésének összefoglalása.

In [ ]:
print("=== GRPO RERANKING ELEMZÉS ÖSSZEFOGLALÓ ===")
print("\n✅ Komponensek állapota:")

if retriever is not None:
    print(f"   🎯 Hybrid retriever: működőképes")
else:
    print(f"   ❌ Hybrid retriever: nem elérhető")

if reranker is not None:
    print(f"   🧠 GRPO reranker: működőképes")
else:
    print(f"   ❌ GRPO reranker: nem elérhető")

if policy_loaded:
    print(f"   📈 RL policy: betöltve")
else:
    print(f"   ⚠️ RL policy: nincs betöltve")

print("\n📋 Agents.md specifikáció ellenőrzés:")
if reranker is not None and policy_loaded:
    print("   ✅ GRPO reranking komponens működőképes")
    print("   🎯 Features: dense similarity, BM25 score, rank difference")
    print("   🏆 Reward: nDCG@10 group level")
    print("   🤖 Policy: linear/shallow MLP")
    print("   📊 Groupwise softmax")
    
    # Teljesítmény ellenőrzés
    try:
        test_query = "szerződés felmondása"
        bm25_results, dense_results = retriever.retrieve_candidates(test_query, top_k=10)
        reranked_results = reranker.rerank(bm25_results, dense_results)
        
        if reranked_results:
            print("   ✅ Reranking működik - eredmények generálva")
            
            # Minőségi metrikák
            scores = [score for _, score in reranked_results[:10]]
            if len(set(scores)) > 1:  # Változatos score-ok
                print("   ✅ Policy differenciál - változatos score-ok")
            else:
                print("   ⚠️ Policy egységes - score-ok nem differenciálnak")
        else:
            print("   ❌ Reranking nem generál eredményeket")
    except Exception as e:
        print(f"   ❌ Reranking működési hiba: {e}")
else:
    missing_components = []
    if retriever is None:
        missing_components.append("Hybrid retriever")
    if reranker is None:
        missing_components.append("GRPO reranker")
    if not policy_loaded:
        missing_components.append("RL policy")
    
    print(f"   ❌ Hiányzó komponensek: {', '.join(missing_components)}")
    print("   💡 Szükséges: uv run courtrankrl build + uv run courtrankrl train")

print("\n💡 Ajánlások:")
if reranker is not None and policy_loaded:
    print("   ✅ GRPO reranking használatra kész")
    print("   🚀 Használat: uv run courtrankrl query \"lekérdezés\" --rerank")
    print("   📈 További javítás: több training data, finomhangolt hyperparameters")
else:
    if not policy_loaded:
        print("   🎓 Policy betanítása szükséges: uv run courtrankrl train")
    print("   🔧 Komponensek inicializálása szükséges")

print("\n🎯 GRPO reranking elemzése kész!")